In [1]:
import pandas as pd
df = pd.read_csv('./data/yellow_tripdata_2015-01_short.csv')

In [2]:
### EXAMPLE WITH A COORDINATE
import geopandas as gpd
from shapely.geometry import Point

barrios = gpd.read_file("./data/barrios_nyc.geojson")

origen = Point(-73.985428, 40.748817)

punto = barrios[barrios.contains(origen)]
print(punto['boro_name'])

63    Manhattan
Name: boro_name, dtype: object


In [3]:
### APPLY TO DATASET
gdf_origin = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['pickup_longitude'], df['pickup_latitude']),
    crs="EPSG:4326" 
)

gdf_origin = gdf_origin.sjoin(barrios, how="left", predicate="intersects")
gdf_origin = gdf_origin.rename(columns={"boro_name": "neigh_origin"})

df['neigh_origin'] = gdf_origin['neigh_origin']

gdf_destination = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['dropoff_longitude'], df['dropoff_latitude']),
    crs="EPSG:4326"
)

gdf_destination = gdf_destination.sjoin(barrios, how="left", predicate="intersects")
gdf_destination = gdf_destination.rename(columns={"boro_name": "neigh_destination"})

df['neigh_destination'] = gdf_destination['neigh_destination']


In [4]:
df[['neigh_origin','neigh_destination']]

,neigh_origin,neigh_destination
0,Manhattan,Manhattan
1,Manhattan,Manhattan
2,Manhattan,Manhattan
3,Manhattan,Manhattan
4,Manhattan,Manhattan
...,...,...
199995,Manhattan,Manhattan
199996,Queens,Brooklyn
199997,Manhattan,Brooklyn
199998,Manhattan,Manhattan


In [6]:
df.to_csv('./data/yellow_tripdata_2015-01_short_neigh.csv') #export as nyc_taxi_neigh.csv